In [2]:
import tkinter as tk
from sympy import *
from IPython.display import Math,display
def solve_optimization_problem(optimize='maximum'):
    var_names = []  
    m = int(entry_variabel.get())
    for i in range(m):
        symbol = entry_vars[i].get()
        var_names.append(symbol)
    var_symbols = symbols(var_names)

    fungsi = sympify(entry_fungsi.get())
    n = int(entry_kendala.get())

    G = 0  
    H = 0  
    g = []  
    h = []  
    c_g = []  
    c_h = []  
    kendala = []  
    tanda = []
    var_g = []
    var_h = []

    for i in range(n):
        func = entry_kendala_funcs[i].get()
        kendala.append(sympify(func))
        tan = entry_kendala_tanda[i].get()
        tanda.append(tan)
        if tanda[i] == '<=':
            var_g.append(symbols(f'lambda_{G + 1}'))
            G += 1
            c_g.append(kendala[i])
            g.append(kendala[i])
        elif tanda[i] == '>=':
            var_g.append(symbols(f'lambda_{G + 1}'))
            G += 1
            c_g.append(-kendala[i])
            g.append(-kendala[i])
        else:
            var_h.append(symbols(f'pi_{H + 1}'))
            H += 1
            c_h.append(kendala[i])
            h.append(kendala[i])

    hesian = hessian(fungsi, var_symbols)
    kondisi = True

    c_g = [c_g[i] * var_g[i] for i in range(G)]
    c_h = [c_h[i] * var_h[i] for i in range(H)]

    if kondisi == True:
        gradf = [diff(fungsi, j) for j in var_symbols]
        gradf = Matrix(gradf)
        
        if H == 0:
            h = zeros(1, 1)
            c_h = zeros(len(var_symbols), 1)
            zero_h = zeros(len(var_symbols), 1)
            zero_h += c_h
        else:
            c_h = Matrix(c_h)
            h = Matrix(h)
            gradh = [diff(i, j) for i in c_h for j in var_symbols]
            gradh = Matrix(gradh).reshape(H, len(var_symbols)).T
            zero_h = zeros(len(var_symbols), 1)
            for i in range(len(gradh[0, :])):
                zero_h += gradh[:, i]

        if G == 0:
            g = zeros(1, 1)
            c_g = zeros(len(var_symbols), 1)
            zero_g = zeros(len(var_symbols), 1)
            zero_g += c_g
        else:
            c_g = Matrix(c_g)
            g = Matrix(g)
            gradg = [diff(i, j) for i in c_g for j in var_symbols]
            gradg = Matrix(gradg).reshape(G, len(var_symbols)).T
            zero_g = zeros(len(var_symbols), 1)
            for i in range(len(gradg[0, :])):
                zero_g += gradg[:, i]

        L = zero_g + zero_h + gradf
        L = L.col_join(c_g)
        L = L.col_join(h)
        L = L.col_join(g)
        gabung = var_g + var_h + var_symbols
        solusi = solve(L, gabung)

        if isinstance(solusi, dict):
            solutions = [solusi]
        else:
            solutions = [dict(zip(gabung, round(i,3))) for i in solusi]

        # Hitung nilai fungsi objektif untuk setiap solusi yang ditemukan
        optimized_values = []
        for sol in solutions:
            if 'lambda' in sol:
                del sol['lambda']  
            obj_func_value = round(fungsi.subs(sol),2)
            optimized_values.append((sol, obj_func_value))

        # Tentukan nilai optimal berdasarkan pilihan (maksimum atau minimum)
        if optimize == 'maximum':
            optimal_solution = max(optimized_values, key=lambda x: x[1])
        else:
            optimal_solution = min(optimized_values, key=lambda x: x[1])

        result_label.config(text=f"Optimal Solution: {optimal_solution[0]}, Optimized Objective Function Value: {optimal_solution[1]}")
    else:
        result_label.config(text="Solusi tidak dapat ditemukan karena definit negatif")

def add_variable_fields():
    global entry_vars, var_entries_frame
    var_entries_frame = tk.Frame(var_frame)
    var_entries_frame.pack()

    entry_vars = []
    var_count = int(entry_variabel.get())
    for i in range(var_count):
        label_var = tk.Label(var_entries_frame, text=f"Masukkan variabel ke-{i + 1}: ")
        label_var.pack()
        entry_var = tk.Entry(var_entries_frame, width=40)  
        entry_var.pack()
        entry_vars.append(entry_var)

def add_constraint_fields():
    global entry_kendala_funcs, entry_kendala_tanda, kendala_entries_frame
    kendala_entries_frame = tk.Frame(constraint_frame)
    kendala_entries_frame.pack()

    entry_kendala_funcs = []
    entry_kendala_tanda = []
    kendala_count = int(entry_kendala.get())
    for i in range(kendala_count):
        label_kendala_func = tk.Label(kendala_entries_frame, text=f"Masukkan fungsi kendala ke-{i + 1}: ")
        label_kendala_func.pack()
        entry_kendala_func = tk.Entry(kendala_entries_frame, width=40)  
        entry_kendala_func.pack()
        entry_kendala_funcs.append(entry_kendala_func)

        label_kendala_tanda = tk.Label(kendala_entries_frame, text=f"Masukkan tanda kendala ke-{i + 1} (<=, >=, =): ")
        label_kendala_tanda.pack()
        entry_kendala_tand = tk.Entry(kendala_entries_frame, width=5)  
        entry_kendala_tand.pack()
        entry_kendala_tanda.append(entry_kendala_tand)
        
def reset_fields():
    global entry_vars, var_entries_frame, entry_kendala_funcs, entry_kendala_tanda, kendala_entries_frame, result_label

    # Hapus isian variabel jika ada
    if 'entry_vars' in globals():
        for entry in entry_vars:
            entry.delete(0, tk.END)
        entry_vars.clear()

    # Hapus isian fungsi objektif dan kendala
    if 'entry_fungsi' in globals():
        entry_fungsi.delete(0, tk.END)
    if 'entry_kendala' in globals():
        entry_kendala.delete(0, tk.END)

    # Hapus frame yang berisi entri untuk variabel jika ada
    if 'var_entries_frame' in globals():
        var_entries_frame.destroy()

    # Hapus frame yang berisi entri untuk kendala jika ada
    if 'kendala_entries_frame' in globals():
        kendala_entries_frame.destroy()

    # Hapus elemen-elemen lain yang perlu direset
    if 'entry_kendala_funcs' in globals():
        entry_kendala_funcs.clear()
    if 'entry_kendala_tanda' in globals():
        entry_kendala_tanda.clear()

    # Hapus hasil yang ditampilkan
    if 'result_label' in globals():
        result_label.config(text="")



def create_gui():
    global root, entry_variabel, entry_kendala, entry_fungsi, var_frame, constraint_frame, result_label

    root = tk.Tk()
    root.title("Program Optimisasi")

    # Konfigurasi warna dan font
    bg_color = "#f0f0f0"  # Warna latar belakang
    fg_color = "#333333"  # Warna teks
    btn_color = "#008CBA"  # Warna tombol
    font_style = ("Arial", 10)  # Gaya font

    root.configure(bg=bg_color)

    var_frame = tk.LabelFrame(root, text="Variabel", bg=bg_color, fg=fg_color, font=font_style)
    var_frame.pack(padx=10, pady=5)

    label_variabel = tk.Label(var_frame, text="Berapa variabel yang digunakan:", bg=bg_color, fg=fg_color, font=font_style)
    label_variabel.pack()

    entry_variabel = tk.Entry(var_frame)
    entry_variabel.pack()

    button_add_var = tk.Button(var_frame, text="Tambah Variabel", command=add_variable_fields, bg=btn_color, fg="white", font=font_style)
    button_add_var.pack()

    constraint_frame = tk.LabelFrame(root, text="Kendala", bg=bg_color, fg=fg_color, font=font_style)
    constraint_frame.pack(padx=10, pady=5)

    label_kendala = tk.Label(constraint_frame, text="Ada berapa kendala:", bg=bg_color, fg=fg_color, font=font_style)
    label_kendala.pack()

    entry_kendala = tk.Entry(constraint_frame)
    entry_kendala.pack()

    button_add_kendala = tk.Button(constraint_frame, text="Tambah Kendala", command=add_constraint_fields, bg=btn_color, fg="white", font=font_style)
    button_add_kendala.pack()

    fungsi_frame = tk.LabelFrame(root, text="Fungsi Objektif", bg=bg_color, fg=fg_color, font=font_style)
    fungsi_frame.pack(padx=10, pady=5)  

    label_fungsi = tk.Label(fungsi_frame, text="Masukkan fungsi objektif:", bg=bg_color, fg=fg_color, font=font_style)
    label_fungsi.pack()

    entry_fungsi = tk.Entry(fungsi_frame)
    entry_fungsi.pack()

    def update_entry_width(event):
        entry_fungsi["width"] = len(entry_fungsi.get())

    entry_fungsi.bind("<KeyRelease>", update_entry_width)

    optimization_choice = tk.StringVar()
    optimization_choice.set("maximum")  # Nilai default untuk optimisasi: maksimum

    optimization_frame = tk.LabelFrame(root, text="Optimization", bg=bg_color, fg=fg_color, font=font_style)
    optimization_frame.pack(padx=10, pady=5)

    max_radio = tk.Radiobutton(optimization_frame, text="Maximize", variable=optimization_choice, value="maximum", bg=bg_color, fg=fg_color, font=font_style)
    max_radio.pack(anchor=tk.W)

    min_radio = tk.Radiobutton(optimization_frame, text="Minimize", variable=optimization_choice, value="minimum", bg=bg_color, fg=fg_color, font=font_style)
    min_radio.pack(anchor=tk.W)

    solve_button = tk.Button(root, text="Solve", command=lambda: solve_optimization_problem(optimization_choice.get()), bg=btn_color, fg="white", font=font_style)
    solve_button.pack(padx=10, pady=10)

    result_label = tk.Label(root, text="", bg=bg_color, fg=fg_color, font=font_style)
    result_label.pack()

    reset_button = tk.Button(root, text="Reset", command=reset_fields, bg=btn_color, fg="white", font=font_style)
    reset_button.pack(padx=10, pady=5)

    root.mainloop()


if __name__ == "__main__":
    create_gui()